In [75]:
import sys
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
# task_idに対応するindexに書き換えてください
start_index = 0
end_index = 499

# ROOT_PATH(出力ファイルを保存するパス)を記載してください
ROOT_PATH = "/content/drive/MyDrive/Colab Notebooks/work/GENIAC/chatbot-arena-ja/"

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


以下のセルを実行し、質問文を（1: そのまま使用可能、2: 修正が必要、3: 削除するべき）でアノテーションしてください。

質問文がよく分からない場合は、"a"と入力すると回答文が表示されます。

In [96]:
import pandas as pd
from datasets import load_dataset
import os

# データセットの読み込み
try:
    dataset_name = "GENIAC-Team-Ozaki/chatbot-arena-ja-calm2-7b-chat-experimental_deduped"
    dataset = load_dataset(dataset_name)
    df = pd.DataFrame(dataset["train"])
except Exception as e:
    print(f"データセットの読み込みに失敗しました: {e}")
    exit()

# Progressファイルのパスを設定
progress_path = os.path.join(ROOT_PATH, "annotation_progress.csv")

def load_progress():
    try:
        progress_df = pd.read_csv(progress_path)
        if not progress_df.empty:
            return progress_df, progress_df['index'].max() + 1
        else:
            return pd.DataFrame(), None
    except FileNotFoundError:
        print("Progressファイルが見つかりません。新しいファイルを作成します。")
        return pd.DataFrame(), None

def annotate_data(df, progress_df, start_index):
    # 範囲をDataFrameに適用
    df = df.loc[start_index:].reset_index(drop=True)
    for index in range(len(df)):
        real_index = start_index + index  # 実際のデータセット内のインデックス
        data = df.loc[index]
        print(f"現在処理中のインデックス: {real_index} \nQuestion: {data['prompt']}")

        while True:
            annotation = input("Annotation (1: そのまま使用可能、2: 修正が必要、3: 削除するべき、a: 回答例の内容表示): ")
            if annotation == 'a':
                print(f"Chosen: {data['chosen']}")
                continue
            if annotation in ["1", "2", "3"]:
                break
            print("無効なアノテーション値です。再入力してください。")

        df.at[index, 'annotation'] = annotation
        new_row = pd.DataFrame({'index': [real_index], 'prompt': [data['prompt']], 'chosen': [data['chosen']],'rejected': [data['rejected']], 'annotation': [annotation]})
        progress_df = pd.concat([progress_df, new_row], ignore_index=True)
        progress_df.to_csv(progress_path, index=False)  # 各ステップごとにProgressを保存

    return df

# Progressファイルを読み込み
progress_df, restart_index = load_progress()
if restart_index is None:
    restart_index = start_index

df = df.loc[restart_index:end_index]

# アノテーションを実施
df_annotated = annotate_data(df, progress_df, restart_index)

# 全アノテーションデータの結合
combined_df = pd.concat([progress_df, df_annotated[df_annotated["annotation"].notna()]], ignore_index=True)

# 処理されたインデックス範囲を含むファイル名を生成
output_filename = os.path.join(ROOT_PATH, f"chatbot_arena_ja_annotated_{start_index}_to_{end_index}.csv")
output_path = output_filename

# outputファイルをCSVとして保存
combined_df.to_csv(output_path, index=False)
print(f"アノテーションデータが {output_filename} に保存されました。")

# Progressファイルを削除
if os.path.exists(progress_path):
    os.remove(progress_path)
    print("Progressファイルが削除されました")


現在処理中のインデックス: 40 
Question: カリフォルニア・スーパーブルームについて、どんなことをご存知ですか?
Annotation (1: そのまま使用可能、2: 修正が必要、3: 削除するべき、a: 回答例の内容表示): 3
現在処理中のインデックス: 41 
Question: エベレスト山の高さはどのくらいですか?


KeyboardInterrupt: Interrupted by user